# Singapore Real-Time Weather Map (Plotly)

This notebook reproduces the core NEA real-time weather workflow from `weather_singapore.ipynb`, but focuses on an interactive Plotly visualization for current air temperature readings across the island.


In [ ]:
import pandas as pd
import requests
import plotly.express as px
from datetime import datetime
from typing import Tuple


In [ ]:
BASE_URL = "https://api-open.data.gov.sg/v2"
AIR_TEMPERATURE_ENDPOINT = "/real-time/api/air-temperature"


def fetch_realtime_air_temperature() -> Tuple[pd.DataFrame, str]:
    """Fetch the latest NEA air temperature readings and return a tidy DataFrame.

    Returns
    -------
    Tuple[pd.DataFrame, str]
        DataFrame of station metadata joined with current temperature values and
        the ISO timestamp supplied by the API.
    """
    response = requests.get(f"{BASE_URL}{AIR_TEMPERATURE_ENDPOINT}", timeout=15)
    response.raise_for_status()
    payload = response.json()

    data_block = payload.get("data", {})
    stations_df = pd.json_normalize(data_block.get("stations", []))
    readings_list = data_block.get("readings", [])
    if not stations_df.size or not readings_list:
        raise ValueError("NEA API returned no station or readings data.")

    readings_df = pd.json_normalize(readings_list[0].get("data", []))
    timestamp = readings_list[0].get("timestamp")

    merged = stations_df.merge(readings_df, left_on="id", right_on="stationId")
    merged = merged.rename(
        columns={
            "location.latitude": "latitude",
            "location.longitude": "longitude",
            "value": "temperature_celsius"
        }
    )

    return merged, timestamp


In [ ]:
stations_df, timestamp_iso = fetch_realtime_air_temperature()

if timestamp_iso:
    timestamp_local = (
        pd.to_datetime(timestamp_iso)
        .tz_localize("UTC")
        .tz_convert("Asia/Singapore")
        .strftime("%Y-%m-%d %H:%M %Z")
    )
else:
    timestamp_local = "Unknown timestamp"

stations_df


In [ ]:
stats = stations_df["temperature_celsius"].agg(["min", "max", "mean"]).round(1)
print(f"NEA data timestamp: {timestamp_local}")
print(
    "Temperature range: "
    f"{stats['min']}°C to {stats['max']}°C (avg {stats['mean']}°C)"
)


In [ ]:
fig = px.scatter_mapbox(
    stations_df,
    lat="latitude",
    lon="longitude",
    color="temperature_celsius",
    size="temperature_celsius",
    size_max=25,
    color_continuous_scale="Turbo",
    hover_name="name",
    hover_data={
        "id": True,
        "temperature_celsius":":.1f",
        "latitude":":.4f",
        "longitude":":.4f",
    },
    zoom=11,
    height=700,
    title=f"NEA Real-Time Air Temperature ({timestamp_local})"
)

fig.update_layout(
    mapbox_style="open-street-map",
    margin=dict(l=10, r=10, t=60, b=10),
    coloraxis_colorbar=dict(title="Temperature (°C)"),
)

fig.show()
